In [ ]:
# Install condacolab
!pip install -q condacolab
import condacolab
condacolab.install()

!conda create -n llava python=3.10 -y

!source activate llava

In [ ]:
!git clone https://github.com/Czi24/LLaVA-20240713.git
%cd /content/LLaVA-20240713/
!pip install -e .

In [ ]:
!pip install huggingface_hub protobuf
%cd /content
from huggingface_hub import snapshot_download
snapshot_download(repo_id="liuhaotian/llava-v1.6-mistral-7b", local_dir="lllava-v1.6-mistral-7b")

In [3]:
import threading
import subprocess

# start Controller
threading.Thread(
    target=lambda: subprocess.run(
        ['python3', '-m', 'llava.serve.controller', '--host', '0.0.0.0', '--port', '10000'],
        check=True
    ),
    daemon=True
).start()

In [4]:
import threading
import subprocess

# Define the command to run the model worker
command = [
    'python3', '-m', 'llava.serve.model_worker',
    '--host', '0.0.0.0',
    '--controller', 'http://localhost:10000',
    '--port', '40000',
    '--worker', 'http://localhost:40000',
    '--model-path', '/content/lllava-v1.6-mistral-7b',
    '--load-8bit'
]

# Run the command in a separate thread
thread = threading.Thread(
    target=lambda: subprocess.run(command, check=True, shell=False),
    daemon=True
)
thread.start()


In [ ]:
!python3 -m llava.serve.gradio_web_server --controller http://localhost:10000 --model-list-mode reload --share